# Skills with Claude API

In [1]:
from dotenv import load_dotenv
import anthropic
from anthropic.lib import files_from_dir

_ = load_dotenv()

In [2]:
client = anthropic.Anthropic()

## Part 1 : Content Creation

### Upload the Custom Skill: Practice Question Generator

In [3]:
# Upload your skill directory
skill = client.beta.skills.create(
    display_title="Practice Question Generator",
    files=files_from_dir("./custom_skills/generating-practice-questions"),
    betas=["skills-2025-10-02"]
)

print(f"Skill ID: {skill.id}")

Skill ID: skill_01Ex1s2aAvVXjHdhYihr5mVV


In [4]:
skills = client.beta.skills.list(source="custom")

In [5]:
for custom_skill in skills:
    print(f"ID: {skill.id}. Display Title: {skill.display_title}")

ID: skill_01Ex1s2aAvVXjHdhYihr5mVV. Display Title: Practice Question Generator


### Upload the Input File 

In [6]:
file_object = client.beta.files.upload(
    file=open("./lecture_notes/notes04.tex", "rb"),
)

### Send the Request

In [7]:
response = client.beta.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=4096,
    betas=["code-execution-2025-08-25","skills-2025-10-02", "files-api-2025-04-14"],  
    container={
        "skills": [{
            "type": "custom", 
            "skill_id": skill.id, 
            "version": "latest"}]
    },
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": "Generate practice questions in Markdown format from these lecture notes"},
            {"type": "container_upload", "file_id": file_object.id}
        ]
    }],
    tools=[{
        "type": "code_execution_20250825",
        "name": "code_execution"
    }]
)

In [8]:
for block in response.content:
    print(block.type)

text
server_tool_use
server_tool_use
text_editor_code_execution_tool_result
text_editor_code_execution_tool_result
text
server_tool_use
text_editor_code_execution_tool_result
text
server_tool_use
text_editor_code_execution_tool_result
text
server_tool_use
bash_code_execution_tool_result
text


In [9]:
# Color codes
BOLD = "\033[1m"
RESET = "\033[0m"
BLUE = "\033[94m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
CYAN = "\033[96m"

In [10]:
for block in response.content:
    if block.type == "text":
        print(f"{BOLD}{BLUE}[TEXT]{RESET}")
        print(block.text, "\n")
    
    if block.type == "server_tool_use":
        print(f"{BOLD}{GREEN}[SERVER_TOOL_USE]{RESET}")
        input_preview = block.input.copy()  
        if "file_text" in input_preview:
            text = input_preview["file_text"]
            if len(text) > 100:
                input_preview["file_text"] = text[:100] + " ..."
        print(input_preview, "\n")
    
    if block.type == "text_editor_code_execution_tool_result":
        print(f"{BOLD}{YELLOW}[TOOL_RESULT]{RESET}")
        content_str = str(block.content)
        preview = content_str[:100]
        if len(content_str) > 100:
            preview += " ..."
        print(preview, "\n")
    
    if block.type == "bash_code_execution_tool_result":
        print(f"{BOLD}{CYAN}[BASH_RESULT]{RESET}")
        print(block.content, "\n")

[TEXT]
I'll help you generate practice questions from the lecture notes. Let me start by reading the skill file and then examining your lecture notes. 

[SERVER_TOOL_USE]
{'command': 'view', 'path': '/skills/generating-practice-questions/SKILL.md'} 

[SERVER_TOOL_USE]
{'command': 'view', 'path': '$INPUT_DIR/notes04.tex'} 

[TOOL_RESULT]
BetaTextEditorCodeExecutionViewResultBlock(content='---\nname: generating-practice-questions\ndescri ... 

[TOOL_RESULT]
BetaTextEditorCodeExecutionViewResultBlock(content="\\documentclass[11pt,letterpaper]{article}\n\\us ... 

[TEXT]
Now let me check the Markdown template to use the proper structure: 

[SERVER_TOOL_USE]
{'command': 'view', 'path': '/skills/generating-practice-questions/assets/markdown_template.md'} 

[TOOL_RESULT]
BetaTextEditorCodeExecutionViewResultBlock(content='# Markdown Template for Practice Questions\n\nUs ... 

[TEXT]
Perfect! Now I'll generate comprehensive practice questions based on the lecture notes about Machine Learning M

### Download the File

In [11]:
# Extract file ID from response
file_id = None
for block in response.content:
    if block.type == 'bash_code_execution_tool_result':
        # File ID is in the tool result
        if hasattr(block, 'content') and hasattr(block.content, 'content'):
            for result_block in block.content.content:
                if hasattr(result_block, 'file_id'):
                    file_id = result_block.file_id
                    break

In [12]:
if file_id:
    # Download the file
    file_content = client.beta.files.download(
        file_id=file_id,
        betas=["files-api-2025-04-14"]
    )

    # Save to disk
    with open("notes04.md", "wb") as f:
        file_content.write_to_file(f.name)

### Delete the Skill - Optional

In [13]:
# Step 1: Delete all versions
versions = client.beta.skills.versions.list(
    skill_id=skill.id,
    betas=["skills-2025-10-02"]
)

for version in versions.data:
    client.beta.skills.versions.delete(
        skill_id=skill.id,
        version=version.version,
        betas=["skills-2025-10-02"]
    )

In [14]:
# Step 2: Delete the Skill
client.beta.skills.delete(
    skill_id=skill.id,
    betas=["skills-2025-10-02"]
)

SkillDeleteResponse(id='skill_01Ex1s2aAvVXjHdhYihr5mVV', type='skill_deleted')

## Part 2 : Data Analysis

### Upload the Custom Skill: Time Series Analysis

In [15]:
# Upload your skill directory
skill = client.beta.skills.create(
    display_title="Time Series Analysis",
    files=files_from_dir("./custom_skills/analyzing-time-series/"),
    betas=["skills-2025-10-02"]
)

print(f"Skill ID: {skill.id}")

Skill ID: skill_01WmT3LwX2Vpc2CYZpahhkoL


In [16]:
skills = client.beta.skills.list()
for sk in skills:
    print(f"ID: {sk.id}. Display Title: {sk.display_title}. Version: {sk.latest_version}")

ID: skill_01WmT3LwX2Vpc2CYZpahhkoL. Display Title: Time Series Analysis. Version: 1768254310772750
ID: xlsx. Display Title: xlsx. Version: 20251013
ID: pptx. Display Title: pptx. Version: 20251013
ID: pdf. Display Title: pdf. Version: 20251013
ID: docx. Display Title: docx. Version: 20251013


### Upload the Input File 

In [17]:
file_object = client.beta.files.upload(
    file=open("./data/retail_sales.csv", "rb"),
)

### Send the Request

In [18]:
query= """
Analyze and visualize this time series data. 
Create a word doc report summarizing the diagnostic results and plots.
"""

In [19]:
response = client.beta.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=16384,
    betas=["code-execution-2025-08-25","skills-2025-10-02", "files-api-2025-04-14"],  
    container={
        "skills": [
            {"type": "custom", "skill_id": skill.id, "version": "latest"},
            {"type": "anthropic", "skill_id":"docx", "version": "20251013"}
        ]
    },
    messages=[{
        "role": "user",
        "content": [
            {"type": "text", "text": query},
            {"type": "container_upload", "file_id": file_object.id}
        ]
    }],
    tools=[{
        "type": "code_execution_20250825",
        "name": "code_execution"
    }]
)

In [20]:
for block in response.content:
    print(block.type)

text
server_tool_use
server_tool_use
text_editor_code_execution_tool_result
text_editor_code_execution_tool_result
text
server_tool_use
bash_code_execution_tool_result
text
server_tool_use
bash_code_execution_tool_result
server_tool_use
bash_code_execution_tool_result
text
server_tool_use
text_editor_code_execution_tool_result
text
server_tool_use
text_editor_code_execution_tool_result
text
server_tool_use
text_editor_code_execution_tool_result
text
server_tool_use
bash_code_execution_tool_result
text
server_tool_use
bash_code_execution_tool_result
text


In [21]:
for block in response.content:
    if block.type == "text":
        print(f"{BOLD}{BLUE}[TEXT]{RESET}")
        print(block.text, "\n")
    
    if block.type == "server_tool_use":
        print(f"{BOLD}{GREEN}[SERVER_TOOL_USE]{RESET}")
        input_preview = block.input.copy()  
        if "file_text" in input_preview:
            text = input_preview["file_text"]
            if len(text) > 100:
                input_preview["file_text"] = text[:100] + " ..."
        print(input_preview, "\n")
    
    if block.type == "text_editor_code_execution_tool_result":
        print(f"{BOLD}{YELLOW}[TOOL_RESULT]{RESET}")
        content_str = str(block.content)
        preview = content_str[:100]
        if len(content_str) > 100:
            preview += " ..."
        print(preview, "\n")
    
    if block.type == "bash_code_execution_tool_result":
        print(f"{BOLD}{CYAN}[BASH_RESULT]{RESET}")
        print(block.content, "\n")

[TEXT]
I'll help you analyze the time series data and create a comprehensive Word document report. Let me start by reading the relevant skill files and then proceeding with the analysis. 

[SERVER_TOOL_USE]
{'command': 'view', 'path': '/skills/analyzing-time-series/SKILL.md'} 

[SERVER_TOOL_USE]
{'command': 'view', 'path': '/skills/docx/SKILL.md'} 

[TOOL_RESULT]
BetaTextEditorCodeExecutionViewResultBlock(content='---\nname: analyzing-time-series\ndescription: C ... 

[TOOL_RESULT]
BetaTextEditorCodeExecutionViewResultBlock(content='---\nname: docx\ndescription: "Comprehensive doc ... 

[TEXT]
Now let me examine the data and run the time series analysis: 

[SERVER_TOOL_USE]
{'command': 'head -20 "$INPUT_DIR/retail_sales.csv"'} 

[BASH_RESULT]
BetaBashCodeExecutionResultBlock(content=[], return_code=0, stderr='', stdout='date,sales\n2015-01-01,46743\n2015-02-01,68008\n2015-03-01,71943\n2015-04-01,62245\n2015-05-01,56679\n2015-06-01,58467\n2015-07-01,46249\n2015-08-01,52259\n2015-09-01,5

### Download the File

In [22]:
# Extract file ID from response
file_id = None
for block in response.content:
    if (block.type == 'bash_code_execution_tool_result'):
        if hasattr(block, 'content') and hasattr(block.content, 'content'):
            for result_block in block.content.content:
                if hasattr(result_block, 'file_id'):
                    file_id = result_block.file_id
                    break

In [23]:
if file_id:
    # Download the file
    file_content = client.beta.files.download(
        file_id=file_id,
        betas=["files-api-2025-04-14"]
    )

    # Save to disk
    with open("report.docx", "wb") as f:
        file_content.write_to_file(f.name)

### Delete the Skill - Optional

In [24]:
# Step 1: Delete all versions
versions = client.beta.skills.versions.list(
    skill_id=skill.id,
    betas=["skills-2025-10-02"]
)

for version in versions.data:
    client.beta.skills.versions.delete(
        skill_id=skill.id,
        version=version.version,
        betas=["skills-2025-10-02"]
    )

In [25]:
# Step 2: Delete the Skill
client.beta.skills.delete(
    skill_id=skill.id,
    betas=["skills-2025-10-02"]
)

SkillDeleteResponse(id='skill_01WmT3LwX2Vpc2CYZpahhkoL', type='skill_deleted')